# Adding Visual Features

In [1]:
%load_ext autoreload
%autoreload 2

import os
os.remove('snorkel.db')

import sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

from snorkel import SnorkelSession
session = SnorkelSession()

from pdb import set_trace as t

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Convert PDF to HTML

Use Adobe Acrobat (or other program of your choice) to convert PDF -> HTML with structure.

### Parse HTML and PDF

In [2]:
import os
from snorkel.parser import CorpusParser
from snorkel.parser import HTMLParser
from snorkel.parser import OmniParser

pdf_path = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware100_pdf/'
html_path = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware100_html/'

filename = 'bc546-d'
html_file = html_path + filename + '.html'

doc_parser = HTMLParser(path=html_file)
context_parser = OmniParser(pdf_path=pdf_path, session=session)
cp = CorpusParser(doc_parser, context_parser, max_docs=100) 

%time corpus = cp.parse_corpus(name='Hardware', session=session)

# Save results
# os.system('cp snorkel.db snorkel.db\ corpus');

Extracted 2690 pdf words
Elapsed: 0.511 s
Extracted 2662 html words
Elapsed: 0.073 s
Linked 2662 words to 2690 bounding boxes
Elapsed: 0.357 s
Updated coordinates in snorkel.db
Elapsed: 1.439 s
[========================================] 100%
CPU times: user 5.21 s, sys: 245 ms, total: 5.46 s
Wall time: 8.09 s


### Load results

In [3]:
# import os
# from snorkel.models import Corpus
# from snorkel.utils import get_ORM_instance

# os.system('cp snorkel.db\ corpus snorkel.db');
# corpus = get_ORM_instance(Corpus, session, 'Hardware')

### Extract Candidates

In [4]:
import os
from collections import defaultdict
from snorkel.models import Corpus, candidate_subclass
from snorkel.matchers import RegexMatchSpan, Union
from snorkel.candidates import CandidateExtractor
from snorkel.utils import get_ORM_instance
from hardware_utils import OmniNgramsPart, OmniNgramsTemp, get_gold_dict

# Candidate Type
Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])
    
# CandidateSpaces
part_ngrams = OmniNgramsPart(parts_by_doc=None, n_max=3) # NOTE: no part linking right now
temp_ngrams = OmniNgramsTemp(n_max=2)

# Matchers
eeca_matcher = RegexMatchSpan(rgx='([b]{1}[abcdefklnpqruyz]{1}[\swxyz]?[0-9]{3,5}[\s]?[A-Z\/]{0,5}[0-9]?[A-Z]?([-][A-Z0-9]{1,7})?([-][A-Z0-9]{1,2})?)')
jedec_matcher = RegexMatchSpan(rgx='([123]N\d{3,4}[A-Z]{0,5}[0-9]?[A-Z]?)')
jis_matcher = RegexMatchSpan(rgx='(2S[abcdefghjkmqrstvz]{1}[\d]{2,4})')
others_matcher = RegexMatchSpan(rgx='((NSVBC|SMBT|MJ|MJE|MPS|MRF|RCA|TIP|ZTX|ZT|TIS|TIPL|DTC|MMBT|PZT){1}[\d]{2,4}[A-Z]{0,3}([-][A-Z0-9]{0,6})?([-][A-Z0-9]{0,1})?)')
parts_matcher = Union(eeca_matcher, jedec_matcher, jis_matcher, others_matcher)

temp_matcher = RegexMatchSpan(rgx=r'1[4-6]0', longest_match_only=False)

# Throttler
part_throttler = lambda x: x[0].parent.page == x[1].parent.page

# Extractor
ce = CandidateExtractor(Part_Temp, 
                        [part_ngrams, temp_ngrams], 
                        [parts_matcher, temp_matcher], 
                        throttler=part_throttler)

# Extract
for corpus_name in ['Hardware']:
    corpus = get_ORM_instance(Corpus, session, corpus_name)
    print "Extracting Candidates from %s" % corpus
    %time candidates = ce.extract(\
        corpus.documents, corpus_name + ' Candidates', session)
    session.add(candidates)
    print "%s contains %d Candidates" % (candidates, len(candidates))
session.commit()

Extracting Candidates from Corpus (Hardware)
[========================================] 100%
CPU times: user 1.4 s, sys: 19.3 ms, total: 1.42 s
Wall time: 1.46 s
Candidate Set (Hardware Candidates) contains 275 Candidates


### View Results

In [5]:
c = candidates[15]
print c.part
print c.part.char_start, c.part.char_end
print c.part.parent.page
print c.part.get_attrib_tokens('top')
print c.part.get_attrib_tokens('bottom')
print c.part.get_attrib_tokens('left')
print c.part.get_attrib_tokens('right')

Span("BC548", parent=1133, chars=[0,4], words=[0,0])
0 4
2
[560]
[569]
[314]
[339]


In [6]:
context_parser.vizlink.display_links(20)

,i,html,pdf,j,offset
0,0,BC546,BC546,42,-5
1,1,-,-,36,-5
2,2,NPN,NPN,13,14
3,3,Amplifier,Amplifier,11,14
4,4,Transistors,Transistors,12,14
5,5,BC546B,BC546B,0,13
6,6,",",",",9,12
7,7,BC547A,BC547A,2,10
8,8,",",",",9,10
9,9,B,B,4,5


In [7]:
# Note: Select image and press any key to close image.
context_parser.vizlink.display_candidates(candidates, page_num=2, display=False)

Boxes per page:
Page 1: 46 (13)
Page 2: 504 (38)


In [12]:
pdf_file = context_parser.vizlink.pdf_file

context_parser.vizlink.display_word('BC546', page_num=2)
# context_parser.vizlink.display_candidates(candidates, page_num=2)

Boxes per page: total (unique)
Page 1: 3 (2)
Page 2: 8 (8)
Page 4: 1 (1)


In [9]:
# from visual_linking import display_boxes, get_box

# boxes = []
# for c in candidates:
#     boxes.append(get_box(c.part))
# boxes = list(set(boxes))
# display_boxes(pdf_file, boxes, page_num=2)

### Display Ordering of PDF Word List 

Display Ordering on a black page - Not very easy to display because of superposition

In [10]:
# import numpy as np
# import cv2
# import math

# page_num = 2
# page_height = 792
# page_width = 612
# img = np.ones((page_height,page_width,3))*255
# font = cv2.FONT_HERSHEY_SIMPLEX
# letter_width = 3
# i = 0
# for word_id, _ in pdf_word_list:
#     if word_id[0] == page_num:
#         i += 1
#         _, top, left, bottom, right = coordinate_map[word_id]
#         cv2.rectangle(img, (left, top), (right, bottom), (0,255,0), 1)
#         cv2.putText(img, 
#                     str(i), 
#                     ((left + right)/2 - letter_width*int(math.ceil(math.log10(i))), 
#                     bottom + (top - bottom)/4), 
#                     font, 
#                     0.3, 
#                     (255,0,0), 
#                     1)
# cv2.imshow('PDF Word List Order',img)
# cv2.waitKey() # press any key to exit the opencv output 
# cv2.destroyAllWindows() 

The end.